In [1]:
import math
import cv2
import numpy as np
import scipy
from scipy import io as scio
from scipy import linalg
from scipy import signal
from scipy import sparse
from skimage.util import img_as_float
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


In [2]:
#QTPIe and multithrads libray in cv 
#pre-processing methods
def process_video(frames):
    """Calculates the average value of each frame."""
    RGB = []
    for frame in frames:
        summation = np.sum(np.sum(frame, axis=0), axis=0)
        RGB.append(summation / (frame.shape[0] * frame.shape[1]))
    return np.asarray(RGB)


def detrend(input_signal, lambda_value):
    signal_length = input_signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = sparse.spdiags(diags_data, diags_index,
                (signal_length - 2), signal_length).toarray()
    filtered_signal = np.dot(
        (H - np.linalg.inv(H + (lambda_value ** 2) * np.dot(D.T, D))), input_signal)
    return filtered_signal

def jade(X, m, Wprev):
    n = X.shape[0]
    T = X.shape[1]
    nem = m
    seuil = 1 / math.sqrt(T) / 100
    if m < n:
        D, U = np.linalg.eig(np.matmul(X, np.mat(X).H) / T)
        Diag = D
        k = np.argsort(Diag)
        pu = Diag[k]
        ibl = np.sqrt(pu[n - m:n] - np.mean(pu[0:n - m]))
        bl = np.true_divide(np.ones(m, 1), ibl)
        W = np.matmul(np.diag(bl), np.transpose(U[0:n, k[n - m:n]]))
        IW = np.matmul(U[0:n, k[n - m:n]], np.diag(ibl))
    else:
        IW = linalg.sqrtm(np.matmul(X, X.H) / T)
        W = np.linalg.inv(IW)

    Y = np.mat(np.matmul(W, X))
    R = np.matmul(Y, Y.H) / T
    C = np.matmul(Y, Y.T) / T
    Q = np.zeros((m * m * m * m, 1))
    index = 0

    for lx in range(m):
        Y1 = Y[lx, :]
        for kx in range(m):
            Yk1 = np.multiply(Y1, np.conj(Y[kx, :]))
            for jx in range(m):
                Yjk1 = np.multiply(Yk1, np.conj(Y[jx, :]))
                for ix in range(m):
                    Q[index] = np.matmul(Yjk1 / math.sqrt(T), Y[ix, :].T / math.sqrt(
                        T)) - R[ix, jx] * R[lx, kx] - R[ix, kx] * R[lx, jx] - C[ix, lx] * np.conj(C[jx, kx])
                    index += 1
    # Compute and Reshape the significant Eigen
    D, U = np.linalg.eig(Q.reshape(m * m, m * m))
    Diag = abs(D)
    K = np.argsort(Diag)
    la = Diag[K]
    M = np.zeros((m, nem * m), dtype=complex)
    Z = np.zeros(m)
    h = m * m - 1
    for u in range(0, nem * m, m):
        Z = U[:, K[h]].reshape((m, m))
        M[:, u:u + m] = la[h] * Z
        h = h - 1
    # Approximate the Diagonalization of the Eigen Matrices:
    B = np.array([[1, 0, 0], [0, 1, 1], [0, 0 - 1j, 0 + 1j]])
    Bt = np.mat(B).H

    encore = 1
    if Wprev == 0:
        V = np.eye(m).astype(complex)
    else:
        V = np.linalg.inv(Wprev)
    # Main Loop:
    while encore:
        encore = 0
        for p in range(m - 1):
            for q in range(p + 1, m):
                Ip = np.arange(p, nem * m, m)
                Iq = np.arange(q, nem * m, m)
                g = np.mat([M[p, Ip] - M[q, Iq], M[p, Iq], M[q, Ip]])
                temp1 = np.matmul(g, g.H)
                temp2 = np.matmul(B, temp1)
                temp = np.matmul(temp2, Bt)
                D, vcp = np.linalg.eig(np.real(temp))
                K = np.argsort(D)
                la = D[K]
                angles = vcp[:, K[2]]
                if angles[0, 0] < 0:
                    angles = -angles
                c = np.sqrt(0.5 + angles[0, 0] / 2)
                s = 0.5 * (angles[1, 0] - 1j * angles[2, 0]) / c

                if abs(s) > seuil:
                    encore = 1
                    pair = [p, q]
                    G = np.mat([[c, -np.conj(s)], [s, c]])  # Givens Rotation
                    V[:, pair] = np.matmul(V[:, pair], G)
                    M[pair, :] = np.matmul(G.H, M[pair, :])
                    temp1 = c * M[:, Ip] + s * M[:, Iq]
                    temp2 = -np.conj(s) * M[:, Ip] + c * M[:, Iq]
                    temp = np.concatenate((temp1, temp2), axis=1)
                    M[:, Ip] = temp1
                    M[:, Iq] = temp2

    # Whiten the Matrix
    # Estimation of the Mixing Matrix and Signal Separation
    A = np.matmul(IW, V)
    S = np.matmul(np.mat(V).H, Y)
    return A, S

def ica(X, Nsources, Wprev=0):
    nRows = X.shape[0]
    nCols = X.shape[1]
    if nRows > nCols:
        print(
            "Warning - The number of rows is cannot be greater than the number of columns.")
        print("Please transpose input.")

    if Nsources > min(nRows, nCols):
        Nsources = min(nRows, nCols)
        print(
            'Warning - The number of soures cannot exceed number of observation channels.')
        print('The number of sources will be reduced to the number of observation channels ', Nsources)

    Winv, Zhat = jade(X, Nsources, Wprev)
    W = np.linalg.pinv(Winv)
    return W, Zhat

def _next_power_of_2(x):
    """Calculate the nearest power of 2."""
    return 1 if x == 0 else 2 ** (x - 1).bit_length()
# Calculate HR
def _calculate_fft_hr(ppg_signal, fs=30, low_pass=0.75, high_pass=2.5):
    """Calculate heart rate based on PPG using Fast Fourier transform (FFT)."""
    ppg_signal = np.expand_dims(ppg_signal, 0)
    N = _next_power_of_2(ppg_signal.shape[1])
    f_ppg, pxx_ppg = scipy.signal.periodogram(ppg_signal, fs=fs, nfft=N, detrend=False)
    fmask_ppg = np.argwhere((f_ppg >= low_pass) & (f_ppg <= high_pass))
    mask_ppg = np.take(f_ppg, fmask_ppg)
    mask_pxx = np.take(pxx_ppg, fmask_ppg)
    fft_hr = np.take(mask_ppg, np.argmax(mask_pxx, 0))[0] * 60
    return fft_hr


In [3]:
#Unsupervised models used

def POS_WANG(frames, fs):
    WinSec = 1.6
    RGB = process_video(frames)
    N = RGB.shape[0]
    H = np.zeros((1, N))
    l = math.ceil(WinSec * fs)

    for n in range(N):
        m = n - l
        if m >= 0:
            Cn = np.true_divide(RGB[m:n, :], np.mean(RGB[m:n, :], axis=0))
            Cn = np.mat(Cn).H
            S = np.matmul(np.array([[0, 1, -1], [-2, 1, 1]]), Cn)
            h = S[0, :] + (np.std(S[0, :]) / np.std(S[1, :])) * S[1, :]
            mean_h = np.mean(h)
            for temp in range(h.shape[1]):
                h[0, temp] = h[0, temp] - mean_h
            H[0, m:n] = H[0, m:n] + (h[0])

    BVP = H
    BVP = detrend(np.mat(BVP).H, 100)
    BVP = np.asarray(np.transpose(BVP))[0]
    b, a = signal.butter(1, [0.75 / fs * 2, 3 / fs * 2], btype='bandpass')
    BVP = signal.filtfilt(b, a, BVP.astype(np.double))
    return BVP

def CHROME_DEHAAN(frames,FS):
    LPF = 0.7
    HPF = 2.5
    WinSec = 1.6

    RGB = process_video(frames)
    FN = RGB.shape[0]
    NyquistF = 1/2*FS
    B, A = signal.butter(3, [LPF/NyquistF, HPF/NyquistF], 'bandpass')

    WinL = math.ceil(WinSec*FS)
    if(WinL % 2):
        WinL = WinL+1
    NWin = math.floor((FN-WinL//2)/(WinL//2))
    WinS = 0
    WinM = int(WinS+WinL//2)
    WinE = WinS+WinL
    totallen = (WinL//2)*(NWin+1)
    S = np.zeros(totallen)

    for i in range(NWin):
        RGBBase = np.mean(RGB[WinS:WinE, :], axis=0)
        RGBNorm = np.zeros((WinE-WinS, 3))
        for temp in range(WinS, WinE):
            RGBNorm[temp-WinS] = np.true_divide(RGB[temp], RGBBase)
        Xs = np.squeeze(3*RGBNorm[:, 0]-2*RGBNorm[:, 1])
        Ys = np.squeeze(1.5*RGBNorm[:, 0]+RGBNorm[:, 1]-1.5*RGBNorm[:, 2])
        Xf = signal.filtfilt(B, A, Xs, axis=0)
        Yf = signal.filtfilt(B, A, Ys)

        Alpha = np.std(Xf) / np.std(Yf)
        SWin = Xf-Alpha*Yf
        SWin = np.multiply(SWin, np.hanning(WinL))

        temp = SWin[:int(WinL//2)]
        S[WinS:WinM] = S[WinS:WinM] + SWin[:int(WinL//2)]
        S[WinM:WinE] = SWin[int(WinL//2):]
        WinS = WinM
        WinM = WinS+WinL//2
        WinE = WinS+WinL
    BVP = S
    return BVP

def ICA_POH(frames, FS):
    # Cut off frequency.
    LPF = 0.7
    HPF = 2.5
    RGB = process_video(frames)

    NyquistF = 1 / 2 * FS
    BGRNorm = np.zeros(RGB.shape)
    Lambda = 100
    for c in range(3):
        BGRDetrend = detrend(RGB[:, c], Lambda)
        BGRNorm[:, c] = (BGRDetrend - np.mean(BGRDetrend)) / np.std(BGRDetrend)
    _, S = ica(np.mat(BGRNorm).H, 3)

    # select BVP Source
    MaxPx = np.zeros((1, 3))
    for c in range(3):
        FF = np.fft.fft(S[c, :])
        F = np.arange(0, FF.shape[1]) / FF.shape[1] * FS * 60
        FF = FF[:, 1:]
        FF = FF[0]
        N = FF.shape[0]
        Px = np.abs(FF[:math.floor(N / 2)])
        Px = np.multiply(Px, Px)
        Fx = np.arange(0, N / 2) / (N / 2) * NyquistF
        Px = Px / np.sum(Px, axis=0)
        MaxPx[0, c] = np.max(Px)
    MaxComp = np.argmax(MaxPx)
    BVP_I = S[MaxComp, :]
    B, A = signal.butter(3, [LPF / NyquistF, HPF / NyquistF], 'bandpass')
    BVP_F = signal.filtfilt(B, A, np.real(BVP_I).astype(np.double))

    BVP = BVP_F[0]
    return BVP

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
df = pd.read_csv("C:/Users/92334/OneDrive/Documents/NUST/FYP/NEW Start/OUT2 FINAL .csv")

# Preprocessing
# Convert string representations of arrays to actual lists of numbers
df['ICA_Output'] = df['ICA_Output'].apply(eval)

# Pad or truncate sequences to a common length
max_length = max(len(seq) for seq in df['ICA_Output'])
X_padded = pad_sequences(df['ICA_Output'].to_list(), maxlen=max_length, padding='post', dtype='float32')

# Use the padded sequences as features
X = np.array(X_padded)

# Split the data into features (ICA_Output) and targets (BP_Sys and BP_Dia)
y_sys = df['BP_Sys']
y_dia = df['BP_Dia']

# Split into training and testing sets
X_train_sys, X_test_sys, y_train_sys, y_test_sys = train_test_split(X, y_sys, test_size=0.3, random_state=42)
X_train_dia, X_test_dia, y_train_dia, y_test_dia = train_test_split(X, y_dia, test_size=0.3, random_state=42)

# Base models with specified parameters
base_models = [
    ('elastic_net', ElasticNet()),
    ('gamma', SVR(kernel='rbf', gamma='scale')),
    ('lasso', Lasso()),
    ('ridge', Ridge()),
    ('random_forest', RandomForestRegressor()),
    ('extra_trees', ExtraTreesRegressor()),
    ('xgboost', XGBRegressor()),
    ('adaboost', AdaBoostRegressor()),
    ('bagging', BaggingRegressor()),
    ('svr_linear', SVR(kernel='linear')),
    ('knn', KNeighborsRegressor()),
    ('mlp', MLPRegressor(hidden_layer_sizes=(100, 50)))
]

# Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the base models and evaluate their performance for systolic blood pressure prediction
mae_scores_sys = {}
for name, model in base_models:
    model.fit(X_train_sys, y_train_sys)
    y_pred_sys = model.predict(X_test_sys)
    mae_sys = mean_absolute_error(y_test_sys, y_pred_sys)
    mae_scores_sys[name] = mae_sys

# Select the top-performing base models for systolic blood pressure prediction
top_models_sys = sorted(mae_scores_sys.items(), key=lambda x: x[1])[:7]
selected_models_sys = [(name, model) for name, _ in top_models_sys]

# Train the base models and evaluate their performance for diastolic blood pressure prediction
mae_scores_dia = {}
for name, model in base_models:
    model.fit(X_train_dia, y_train_dia)
    y_pred_dia = model.predict(X_test_dia)
    mae_dia = mean_absolute_error(y_test_dia, y_pred_dia)
    mae_scores_dia[name] = mae_dia

# Select the top-performing base models for diastolic blood pressure prediction
top_models_dia = sorted(mae_scores_dia.items(), key=lambda x: x[1])[:7]
selected_models_dia = [(name, model) for name, _ in top_models_dia]

# Stacked ensemble model for systolic blood pressure prediction
stacked_model_sys = StackingRegressor(estimators=selected_models_sys, final_estimator=LinearRegression())
stacked_model_sys.fit(X_train_sys, y_train_sys)
y_pred_stacked_sys = stacked_model_sys.predict(X_test_sys)
mae_stacked_sys = mean_absolute_error(y_test_sys, y_pred_stacked_sys)
print('Stacked Ensemble (Systolic BP) Mean Absolute Error:', mae_stacked_sys)

# Stacked ensemble model for diastolic blood pressure prediction
stacked_model_dia = StackingRegressor(estimators=selected_models_dia, final_estimator=LinearRegression())
stacked_model_dia.fit(X_train_dia, y_train_dia)
y_pred_stacked_dia = stacked_model_dia.predict(X_test_dia)
mae_stacked_dia = mean_absolute_error(y_test_dia, y_pred_stacked_dia)
print('Stacked Ensemble (Diastolic BP) Mean Absolute Error:', mae_stacked_dia)


c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

Stacked Ensemble (Systolic BP) Mean Absolute Error: 16.508883992179495


c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

Stacked Ensemble (Diastolic BP) Mean Absolute Error: 8.728010959312565


c:\Users\92334\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
print(X[0])

[ 4.84397560e-02  1.18322805e-01  1.78405479e-01  2.21302852e-01
  2.43450567e-01  2.45532602e-01  2.31848225e-01  2.08830327e-01
  1.83286190e-01  1.60795853e-01  1.44556791e-01  1.34977862e-01
  1.29941970e-01  1.25467077e-01  1.16678283e-01  9.88625363e-02
  6.83714077e-02  2.34080497e-02 -3.53981704e-02 -1.04799338e-01
 -1.79089531e-01 -2.50705391e-01 -3.11313480e-01 -3.53095204e-01
 -3.70006621e-01 -3.58859450e-01 -3.19956899e-01 -2.57250190e-01
 -1.78027019e-01 -9.19107199e-02 -9.22734383e-03  6.08742237e-02
  1.12158015e-01  1.42362997e-01  1.53242603e-01  1.49740726e-01
  1.38527587e-01  1.26206189e-01  1.17783144e-01  1.15752637e-01
  1.19850494e-01  1.27536327e-01  1.34970218e-01  1.38110295e-01
  1.33783072e-01  1.20483726e-01  9.86525789e-02  7.04841465e-02
  3.93232964e-02  8.77438579e-03 -1.80688687e-02 -3.90760154e-02
 -5.32407537e-02 -6.04969077e-02 -6.13738894e-02 -5.67600429e-02
 -4.77293693e-02 -3.54488976e-02 -2.12501213e-02 -6.70096511e-03
  6.37195865e-03  1.58568

In [6]:
from sklearn.metrics import mean_squared_error

# Calculate mean error (ME) for systolic blood pressure prediction
me_stacked_sys = np.mean(y_pred_stacked_sys - y_test_sys)

# Calculate standard deviation (SD) for systolic blood pressure prediction
sd_stacked_sys = np.std(y_pred_stacked_sys - y_test_sys)

# Calculate mean absolute error (MAE) for diastolic blood pressure prediction
mae_stacked_dia = mean_absolute_error(y_test_dia, y_pred_stacked_dia)

# Calculate mean error (ME) for diastolic blood pressure prediction
me_stacked_dia = np.mean(y_pred_stacked_dia - y_test_dia)

# Calculate standard deviation (SD) for diastolic blood pressure prediction
sd_stacked_dia = np.std(y_pred_stacked_dia - y_test_dia)

# Print the results
print('Stacked Ensemble (Systolic BP) Mean Error:', me_stacked_sys)
print('Stacked Ensemble (Systolic BP) Standard Deviation:', sd_stacked_sys)
print('Stacked Ensemble (Systolic BP) Mean Absolute Error:', mae_stacked_sys)

print('Stacked Ensemble (Diastolic BP) Mean Error:', me_stacked_dia)
print('Stacked Ensemble (Diastolic BP) Standard Deviation:', sd_stacked_dia)
print('Stacked Ensemble (Diastolic BP) Mean Absolute Error:', mae_stacked_dia)


Stacked Ensemble (Systolic BP) Mean Error: -3.542430815149526
Stacked Ensemble (Systolic BP) Standard Deviation: 22.22269526710181
Stacked Ensemble (Systolic BP) Mean Absolute Error: 16.484512704317687
Stacked Ensemble (Diastolic BP) Mean Error: 2.1916051145459785
Stacked Ensemble (Diastolic BP) Standard Deviation: 11.49076703186583
Stacked Ensemble (Diastolic BP) Mean Absolute Error: 9.318551235511654


In [14]:
print(type(y_pred_stacked_sys))

<class 'numpy.ndarray'>


In [13]:
#only haarcascade
import cv2

# Load pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create an array to store frames
frames_array = []

# Set the maximum number of frames
max_frames = 900
FS=30

# Open webcam
cap = cv2.VideoCapture(0)

while len(frames_array) < max_frames:
    # Read a frame from the webcam
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Display the frame with the rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Store the face in the array
        frames_array.append(face_roi)
 
    cv2.putText(frame, f"Captured frames: {len(frames_array)}/{max_frames}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Capturing Video for Blood Pressure", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(frame, "and Heart Rate!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

# Create a black screen with the length of the frames array number
final_screen = np.zeros((1000, 100 * len(frames_array), 3), dtype=np.uint8)
ppg_valuesICA=ICA_POH(frames_array,FS)
ppg_valuesPOS=POS_WANG(frames_array,FS)
ppg_valuesCHROM=CHROME_DEHAAN(frames_array,FS)

heart_rateICA=_calculate_fft_hr(ppg_valuesICA)
heart_ratePOS=_calculate_fft_hr(ppg_valuesPOS)
heart_rateCHROM=_calculate_fft_hr(ppg_valuesCHROM)

reshaped_ppgPOS = ppg_valuesPOS.reshape(1, -1)
reshaped_ppgICA = ppg_valuesICA.reshape(1, -1)
reshaped_ppgCHROM = ppg_valuesCHROM.reshape(1, -1)

y_pred_sysICA = stacked_model_sys.predict(reshaped_ppgICA)
y_pred_diaICA = stacked_model_dia.predict(reshaped_ppgICA)
y_pred_sysPOS = stacked_model_sys.predict(reshaped_ppgPOS)
y_pred_diaPOS = stacked_model_dia.predict(reshaped_ppgPOS)
# y_pred_sysCHROME = stacked_model_sys.predict(reshaped_ppgCHROM)
# y_pred_diaCHROME = stacked_model_dia.predict(reshaped_ppgCHROM)


cv2.putText(final_screen, f"Number of Frames: {len(frames_array)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate ICA: {heart_rateICA}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate POS: {heart_ratePOS}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate CHROME: {heart_rateCHROM}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure ICA: {y_pred_sysICA[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure ICA: {y_pred_diaICA[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure POS: {y_pred_sysPOS[0]}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure POS: {y_pred_diaPOS[0]}", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# cv2.putText(final_screen, f"Calculated Systolic Blood Pressure CHROME: {y_pred_sysCHROME[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
# cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure CHROME: {y_pred_diaCHROME[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Display the black screen
cv2.imshow('Final Screen', final_screen)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
#Countour segmentation

import cv2

# Load pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create an array to store frames
frames_array = []

# Set the maximum number of frames
max_frames = 900

# Open webcam
cap = cv2.VideoCapture(0)

while len(frames_array) < max_frames:
    # Read a frame from the webcam
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Draw rectangles around the detected faces and apply contour detection
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Convert the face region to grayscale
        face_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred_face = cv2.GaussianBlur(face_gray, (5, 5), 0)

        # Apply thresholding to obtain binary image
        _, thresh = cv2.threshold(blurred_face, 90, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours on the original face region
        cv2.drawContours(face_roi, contours, -1, (0, 255, 0), 2)

        # Display the frame with the rectangle around the face and contours
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Store the face in the array
        frames_array.append(face_roi)

    cv2.putText(frame, f"Captured frames: {len(frames_array)}/{max_frames}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Capturing Video for Blood Pressure", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(frame, "and Heart Rate!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


# Create a black screen with the length of the frames array number
final_screen = np.zeros((1000, 100 * len(frames_array), 3), dtype=np.uint8)
ppg_valuesICA=ICA_POH(frames_array,FS)
ppg_valuesPOS=POS_WANG(frames_array,FS)
ppg_valuesCHROM=CHROME_DEHAAN(frames_array,FS)

heart_rateICA=_calculate_fft_hr(ppg_valuesICA)
heart_ratePOS=_calculate_fft_hr(ppg_valuesPOS)
heart_rateCHROM=_calculate_fft_hr(ppg_valuesCHROM)

reshaped_ppgPOS = ppg_valuesPOS.reshape(1, -1)
reshaped_ppgICA = ppg_valuesICA.reshape(1, -1)
reshaped_ppgCHROM = ppg_valuesCHROM.reshape(1, -1)

y_pred_sysICA = stacked_model_sys.predict(reshaped_ppgICA)
y_pred_diaICA = stacked_model_dia.predict(reshaped_ppgICA)
y_pred_sysPOS = stacked_model_sys.predict(reshaped_ppgPOS)
y_pred_diaPOS = stacked_model_dia.predict(reshaped_ppgPOS)
# y_pred_sysCHROME = stacked_model_sys.predict(reshaped_ppgCHROM)
# y_pred_diaCHROME = stacked_model_dia.predict(reshaped_ppgCHROM)


cv2.putText(final_screen, f"Number of Frames: {len(frames_array)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate ICA: {heart_rateICA}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate POS: {heart_ratePOS}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate CHROME: {heart_rateCHROM}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure ICA: {y_pred_sysICA[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure ICA: {y_pred_diaICA[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure POS: {y_pred_sysPOS[0]}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure POS: {y_pred_diaPOS[0]}", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# cv2.putText(final_screen, f"Calculated Systolic Blood Pressure CHROME: {y_pred_sysCHROME[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
# cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure CHROME: {y_pred_diaCHROME[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Display the black screen
cv2.imshow('Final Screen', final_screen)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: X has 215 features, but MLPRegressor is expecting 900 features as input.

In [19]:
import cv2
import numpy as np
import torch
from models import FaceNetModel  # Assuming you have a FaceNet model class

# Load pre-trained FaceNet model
model = FaceNetModel()
model.load_state_dict(torch.load('C:/Users/92334/Downloads/model_resnet34_triplet.pt'))
model.eval()

# Create an array to store frames
frames_array = []

# Set the maximum number of frames
max_frames = 900
FS = 30

# Open webcam
cap = cv2.VideoCapture(0)

while len(frames_array) < max_frames:
    # Read a frame from the webcam
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the FaceNet model
    # Assuming FaceNetModel has a method detect_faces which returns bounding boxes of faces
    faces = model.detect_faces(frame)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Display the frame with the rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Store the face in the array
        frames_array.append(face_roi)

    cv2.putText(frame, f"Captured frames: {len(frames_array)}/{max_frames}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Capturing Video for Blood Pressure", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(frame, "and Heart Rate!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

final_screen = np.zeros((1000, 100 * len(frames_array), 3), dtype=np.uint8)
ppg_valuesICA=ICA_POH(frames_array,FS)
ppg_valuesPOS=POS_WANG(frames_array,FS)
ppg_valuesCHROM=CHROME_DEHAAN(frames_array,FS)

heart_rateICA=_calculate_fft_hr(ppg_valuesICA)
heart_ratePOS=_calculate_fft_hr(ppg_valuesPOS)
heart_rateCHROM=_calculate_fft_hr(ppg_valuesCHROM)

reshaped_ppgPOS = ppg_valuesPOS.reshape(1, -1)
reshaped_ppgICA = ppg_valuesICA.reshape(1, -1)
reshaped_ppgCHROM = ppg_valuesCHROM.reshape(1, -1)

y_pred_sysICA = stacked_model_sys.predict(reshaped_ppgICA)
y_pred_diaICA = stacked_model_dia.predict(reshaped_ppgICA)
y_pred_sysPOS = stacked_model_sys.predict(reshaped_ppgPOS)
y_pred_diaPOS = stacked_model_dia.predict(reshaped_ppgPOS)
# y_pred_sysCHROME = stacked_model_sys.predict(reshaped_ppgCHROM)
# y_pred_diaCHROME = stacked_model_dia.predict(reshaped_ppgCHROM)


cv2.putText(final_screen, f"Number of Frames: {len(frames_array)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate ICA: {heart_rateICA}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate POS: {heart_ratePOS}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Heart Rate CHROME: {heart_rateCHROM}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure ICA: {y_pred_sysICA[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure ICA: {y_pred_diaICA[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.putText(final_screen, f"Calculated Systolic Blood Pressure POS: {y_pred_sysPOS[0]}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure POS: {y_pred_diaPOS[0]}", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# cv2.putText(final_screen, f"Calculated Systolic Blood Pressure CHROME: {y_pred_sysCHROME[0]}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
# cv2.putText(final_screen, f"Calculated Diastolic Blood Pressure CHROME: {y_pred_diaCHROME[0]}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Display the black screen
cv2.imshow('Final Screen', final_screen)
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'models'